# Fetch Articles Abstracts from Scopus Search API

### by Miguel Silva Brito

### Dec, 2017

First you need an API KEY from Scopus, get it from https://dev.elsevier.com

Remember to connect to your University VPN first!


In [ ]:
API_KEY=""##### <<<<<<<< CHANGE HERE!!!!

In [136]:
#max number of results to fetch each time depending of Service Level in Scopus API 
COUNT=5

Your query in here

In [137]:
QUERY="MACHINE LEARNING FORECAST DEMAND "

In [138]:
import requests
import json
import pandas as pd

### Search request to Scopus API

In [139]:
def run_scopus_search(start):
    resp = requests.get("http://api.elsevier.com/content/search/scopus?start=%s&count=%s&query=%s" % (str(start),str(COUNT),QUERY),
                        headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    print("fetching Scopus IDs... %d " % start)
    results= json.loads(resp.text.encode('utf-8'))
    return results

### Return scopus IDs from results 

In [140]:
def get_scopus_ids(results):
    return [[str(r['dc:identifier'])] for r in results['search-results']["entry"]]

### Return total numbers of articles found 

In [141]:
def get_totalResults(results):
    return int(results['search-results']['opensearch:totalResults'])

### Return basic info about article

In [142]:
def get_scopus_info(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/abstract/scopus_id/"+ SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})

    return json.loads(resp.text.encode('utf-8'))

### Return abstract article

In [143]:
def get_scopus_text(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/article/scopus_id/" + SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    return json.loads(resp.text.encode('utf-8'))

because fetching results could take hours we store results in base file in case the process is interrupt, you can start it again without fetch again the abstracts

In [144]:
from pathlib import Path
number_articles_allready_load=0
base = Path("./data/scopus_search_base.csv")
base_exist=False
new_base_exist=False
if base.is_file():
    article_set_readed=pd.read_csv("./data/scopus_search_base.csv")
    base_exist=True
    number_articles_allready_load=article_set.shape[0]
    print('base exists ,%d articles '% number_articles_allready_load) 
new_base_exist=base_exist

### Main

run scopus search, return top 200 and number of total articles
loop by next 200 until reach total articles

write dataset to csv file

In [ ]:
start=0
article_data=[]

n=0

results=run_scopus_search(start)
totalResults=get_totalResults(results)

scopus_ids=get_scopus_ids(results)
if base_exist==True:
    scopus_idsdf=pd.DataFrame(scopus_ids)
    scopus_ids=scopus_idsdf[~scopus_idsdf.isin(article_set_readed['Scopus_ID'])].dropna(how='all').values

n_ids_fetched =len(scopus_ids)
print("number of scopus ids fetched : %d de %d" % (n_ids_fetched,totalResults))

while (n<totalResults-number_articles_allready_load):
    nn=0
    for scopus_id in scopus_ids:

        results_info = get_scopus_info(scopus_id[0])
        results_text = get_scopus_text(scopus_id[0])
        try:
            title=results_info['abstracts-retrieval-response']['coredata']['dc:title']
            pubDate=results_info['abstracts-retrieval-response']['coredata']['prism:coverDate']
            try:
                abstract=str(results_text['full-text-retrieval-response']['coredata']['dc:description'])
                abstract=abstract.replace('Abstract\n               \n                  ','')
            except:
                abstract='(not available)'
            print("fetching %d / %d..." % (n+1,totalResults)) 
            n=n+1
            nn=nn+1
            author=[[str(row['ce:indexed-name'])] for row in results_info['abstracts-retrieval-response']['coredata']['dc:creator']['author']]
            article_data.append({'Scopus_ID' : scopus_id[0],'title':title,'pubDate':pubDate,'abstract': abstract,'author': author[0][0]})
        except:
            pass
    if nn>0:
        article_set=pd.DataFrame(article_data)
        article_data=[]
        if (new_base_exist==False):
            print('creating base')
            article_set.to_csv("./data/scopus_search_base.csv",index=False)
            new_base_exist=True
        else:
            print('append to base')
            article_set.to_csv("./data/scopus_search_base.csv",mode="a",header=False,index=False)
           
    start=start+n_ids_fetched
    results=run_scopus_search(start)
    scopus_ids=get_scopus_ids(results)
    n_ids_fetched =len(scopus_ids)
    if base_exist==True:
        scopus_idsdf=pd.DataFrame(scopus_ids)
        scopus_ids=scopus_idsdf[~scopus_idsdf.isin(article_set_readed['Scopus_ID'])].dropna(how='all').values
        print('novos artigos= %d %d' % (len(scopus_ids),article_set_readed.shape[0]))
    print("number of scopus ids fetched : %d de %d" % (n_ids_fetched+start,totalResults))
print("Finish!")

fetching Scopus IDs... 0 
number of scopus ids fetched : 5 de 4600
fetching 1 / 4600...
fetching 2 / 4600...
fetching 3 / 4600...
fetching 4 / 4600...
fetching 5 / 4600...
creating base
fetching Scopus IDs... 5 
number of scopus ids fetched : 10 de 4600
fetching 6 / 4600...
fetching 7 / 4600...
fetching 8 / 4600...
fetching 9 / 4600...
fetching 10 / 4600...
append to base
fetching Scopus IDs... 10 
number of scopus ids fetched : 15 de 4600
fetching 11 / 4600...
fetching 12 / 4600...
fetching 13 / 4600...
fetching 14 / 4600...
fetching 15 / 4600...
append to base
fetching Scopus IDs... 15 
number of scopus ids fetched : 20 de 4600
fetching 16 / 4600...
fetching 17 / 4600...
fetching 18 / 4600...
fetching 19 / 4600...
fetching 20 / 4600...
append to base
fetching Scopus IDs... 20 
number of scopus ids fetched : 25 de 4600
fetching 21 / 4600...
fetching 22 / 4600...
fetching 23 / 4600...
fetching 24 / 4600...
fetching 25 / 4600...
append to base
fetching Scopus IDs... 25 
number of scopus

fetching 208 / 4600...
fetching 209 / 4600...
fetching 210 / 4600...
append to base
fetching Scopus IDs... 210 
number of scopus ids fetched : 215 de 4600
fetching 211 / 4600...
fetching 212 / 4600...
fetching 213 / 4600...
fetching 214 / 4600...
fetching 215 / 4600...
append to base
fetching Scopus IDs... 215 
number of scopus ids fetched : 220 de 4600
fetching 216 / 4600...
fetching 217 / 4600...
fetching 218 / 4600...
fetching 219 / 4600...
fetching 220 / 4600...
append to base
fetching Scopus IDs... 220 
number of scopus ids fetched : 225 de 4600
fetching 221 / 4600...
fetching 222 / 4600...
fetching 223 / 4600...
fetching 224 / 4600...
fetching 225 / 4600...
append to base
fetching Scopus IDs... 225 
number of scopus ids fetched : 230 de 4600
fetching 226 / 4600...
fetching 227 / 4600...
fetching 228 / 4600...
fetching 229 / 4600...
fetching 230 / 4600...
append to base
fetching Scopus IDs... 230 
number of scopus ids fetched : 235 de 4600
fetching 231 / 4600...
fetching 232 / 460

fetching 411 / 4600...
fetching 412 / 4600...
fetching 413 / 4600...
fetching 414 / 4600...
append to base
fetching Scopus IDs... 415 
number of scopus ids fetched : 420 de 4600
fetching 415 / 4600...
fetching 416 / 4600...
fetching 417 / 4600...
fetching 418 / 4600...
fetching 419 / 4600...
append to base
fetching Scopus IDs... 420 
number of scopus ids fetched : 425 de 4600
fetching 420 / 4600...
fetching 421 / 4600...
fetching 422 / 4600...
fetching 423 / 4600...
fetching 424 / 4600...
append to base
fetching Scopus IDs... 425 
number of scopus ids fetched : 430 de 4600
fetching 425 / 4600...
fetching 426 / 4600...
fetching 427 / 4600...
fetching 428 / 4600...
fetching 429 / 4600...
append to base
fetching Scopus IDs... 430 
number of scopus ids fetched : 435 de 4600
fetching 430 / 4600...
fetching 431 / 4600...
fetching 432 / 4600...
fetching 433 / 4600...
fetching 434 / 4600...
append to base
fetching Scopus IDs... 435 
number of scopus ids fetched : 440 de 4600
fetching 435 / 460

fetching 615 / 4600...
fetching 616 / 4600...
fetching 617 / 4600...
fetching 618 / 4600...
fetching 619 / 4600...
append to base
fetching Scopus IDs... 620 
number of scopus ids fetched : 625 de 4600
fetching 620 / 4600...
fetching 621 / 4600...
fetching 622 / 4600...
fetching 623 / 4600...
fetching 624 / 4600...
append to base
fetching Scopus IDs... 625 
number of scopus ids fetched : 630 de 4600
fetching 625 / 4600...
fetching 626 / 4600...
fetching 627 / 4600...
fetching 628 / 4600...
fetching 629 / 4600...
append to base
fetching Scopus IDs... 630 
number of scopus ids fetched : 635 de 4600
fetching 630 / 4600...
fetching 631 / 4600...
fetching 632 / 4600...
fetching 633 / 4600...
fetching 634 / 4600...
append to base
fetching Scopus IDs... 635 
number of scopus ids fetched : 640 de 4600
fetching 635 / 4600...
fetching 636 / 4600...
fetching 637 / 4600...
fetching 638 / 4600...
fetching 639 / 4600...
append to base
fetching Scopus IDs... 640 
number of scopus ids fetched : 645 de 

fetching 820 / 4600...
fetching 821 / 4600...
fetching 822 / 4600...
fetching 823 / 4600...
fetching 824 / 4600...
append to base
fetching Scopus IDs... 825 
number of scopus ids fetched : 830 de 4600
fetching 825 / 4600...
fetching 826 / 4600...
fetching 827 / 4600...
fetching 828 / 4600...
fetching 829 / 4600...
append to base
fetching Scopus IDs... 830 
number of scopus ids fetched : 835 de 4600
fetching 830 / 4600...
fetching 831 / 4600...
fetching 832 / 4600...
fetching 833 / 4600...
fetching 834 / 4600...
append to base
fetching Scopus IDs... 835 
number of scopus ids fetched : 840 de 4600
fetching 835 / 4600...
fetching 836 / 4600...
fetching 837 / 4600...
fetching 838 / 4600...
fetching 839 / 4600...
append to base
fetching Scopus IDs... 840 
number of scopus ids fetched : 845 de 4600
fetching 840 / 4600...
fetching 841 / 4600...
fetching 842 / 4600...
fetching 843 / 4600...
fetching 844 / 4600...
append to base
fetching Scopus IDs... 845 
number of scopus ids fetched : 850 de 

fetching Scopus IDs... 1025 
number of scopus ids fetched : 1030 de 4600
fetching 1025 / 4600...
fetching 1026 / 4600...
fetching 1027 / 4600...
fetching 1028 / 4600...
fetching 1029 / 4600...
append to base
fetching Scopus IDs... 1030 
number of scopus ids fetched : 1035 de 4600
fetching 1030 / 4600...
fetching 1031 / 4600...
fetching 1032 / 4600...
fetching 1033 / 4600...
fetching 1034 / 4600...
append to base
fetching Scopus IDs... 1035 
number of scopus ids fetched : 1040 de 4600
fetching 1035 / 4600...
fetching 1036 / 4600...
fetching 1037 / 4600...
fetching 1038 / 4600...
fetching 1039 / 4600...
append to base
fetching Scopus IDs... 1040 
number of scopus ids fetched : 1045 de 4600
fetching 1040 / 4600...
fetching 1041 / 4600...
fetching 1042 / 4600...
fetching 1043 / 4600...
fetching 1044 / 4600...
append to base
fetching Scopus IDs... 1045 
number of scopus ids fetched : 1050 de 4600
fetching 1045 / 4600...
fetching 1046 / 4600...
fetching 1047 / 4600...
fetching 1048 / 4600...

fetching 1221 / 4600...
fetching 1222 / 4600...
fetching 1223 / 4600...
fetching 1224 / 4600...
append to base
fetching Scopus IDs... 1225 
number of scopus ids fetched : 1230 de 4600
fetching 1225 / 4600...
fetching 1226 / 4600...
fetching 1227 / 4600...
fetching 1228 / 4600...
fetching 1229 / 4600...
append to base
fetching Scopus IDs... 1230 
number of scopus ids fetched : 1235 de 4600
fetching 1230 / 4600...
fetching 1231 / 4600...
fetching 1232 / 4600...
fetching 1233 / 4600...
fetching 1234 / 4600...
append to base
fetching Scopus IDs... 1235 
number of scopus ids fetched : 1240 de 4600
fetching 1235 / 4600...
fetching 1236 / 4600...
fetching 1237 / 4600...
fetching 1238 / 4600...
fetching 1239 / 4600...
append to base
fetching Scopus IDs... 1240 
number of scopus ids fetched : 1245 de 4600
fetching 1240 / 4600...
fetching 1241 / 4600...
fetching 1242 / 4600...
fetching 1243 / 4600...
fetching 1244 / 4600...
append to base
fetching Scopus IDs... 1245 
number of scopus ids fetched

In [ ]:
article_set=pd.read_csv("./data/scopus_search_base.csv") 
print(article_set.info)
print(article_set.head)